In [1]:
import nltk.corpus
import string
import os
import sys
#os.chdir("/home/tanya/Desktop/major2")
os.curdir
if 'SPARK_HOME' not in os.environ:
    os.environ['SPARK_HOME']='c/spark-2.0.0-bin-hadoop2.7'
    #os.environ['SPARK_HOME']='/home/garima/spark-2.0.0-bin-hadoop2.7'
SPARK_HOME=os.environ['SPARK_HOME']

sys.path.insert(0,os.path.join(SPARK_HOME,"python"))
sys.path.insert(0,os.path.join(SPARK_HOME,"python","lib"))
sys.path.insert(0,os.path.join(SPARK_HOME,"python","lib","pyspark.zip"))
sys.path.insert(0,os.path.join(SPARK_HOME,"python","lib","py4j"))
#sys.path.insert(0,os.path.join(SPARK_HOME,"python","lib","py4j-0.10.1-src.zip"))
from pyspark import SparkContext
from pyspark import SparkConf

conf=SparkConf()
conf.set("spark.executor.memory","1g")
conf.set("spark.cores.max","1")

conf.setAppName("major")

sc = SparkContext('local',conf=conf)


In [2]:
from pyspark.sql import SQLContext
from pyspark.sql import Row
import nltk.corpus
import string
import re , math
from collections import Counter

In [3]:
def fil(x):
    l = x.split(',')
    statusemo=0
    
    if(len(l)>=4):
        sid = list(str(l[1]))[5:]
        sid = int(''.join(sid))
        meanEmo=[]
        posorneg=[]
        emocode=[]
        outset=[]
        #convert to lower case
        text = str(l[3].decode('utf-8'))
        text=text.replace('"text":', '')
        bhasad=text
        #bhasad=bhasad.replace('"', '')
        #textstr = (l[3].encode('utf-8').decode('unicode-escape')).encode().lower()
        textemo = l[3].encode('utf-8').decode('unicode-escape')
        #textemoparttwo = bhasad.encode('utf-8').decode('unicode-escape')
        textemoparttwo= unicode(bhasad,'unicode-escape')
        a=textemoparttwo.encode('utf-8')
        emoticonsoveralllist=[[1,'\xF0\x9F\x98\x81','U+1F601','GRINNING FACE WITH SMILING EYES','happy','positive'],
                              [2,'\xF0\x9F\x98\x82','U+1F602','FACE WITH TEARS OF JOY','happy','positive'],
                              [3,'\xF0\x9F\x98\x83','U+1F603','SMILING FACE WITH OPEN MOUTH','happy','positive'],
                              [4,'\xF0\x9F\x98\x84','U+1F604','SMILING FACE WITH OPEN MOUTH AND SMILING EYES','happy','positive'],
                              [5,'\xF0\x9F\x98\x85','U+1F605','SMILING FACE WITH OPEN MOUTH AND COLD SWEAT','happy','positive'],
                              [6,'\xF0\x9F\x98\x86','U+1F606','SMILING FACE WITH OPEN MOUTH AND TIGHTLY-CLOSED EYES','happy','positive'],
                              [7,'\xF0\x9F\x98\x89','U+1F609','WINKING FACE','happy','sarcastic'],
                              [8,'\xF0\x9F\x98\x8A','U+1F60A','SMILING FACE WITH SMILING EYES','happy','positive'],
                              [9,'\xF0\x9F\x98\x8B','U+1F60B','FACE SAVOURING DELICIOUS FOOD','happy','positive'],
                              [10,'\xF0\x9F\x98\x8C','U+1F60C','RELIEVED FACE','happy','positive'],
                              [11,'\xF0\x9F\x98\x8D','U+1F60D','SMILING FACE WITH HEART-SHAPED EYES','happy','positive'],
                              [12,'\xF0\x9F\x98\x8F','U+1F60F','SMIRKING FACE','sad','negative'],
                              [13,'\xF0\x9F\x98\x92','U+1F612','UNAMUSED FACE','sad','negative'],
                              [14,'\xF0\x9F\x98\x93','U+1F613','FACE WITH COLD SWEAT','sad','negative'],
                              [15,'\xF0\x9F\x98\x94','U+1F614','PENSIVE FACE','sad','negative'],
                              [16,'\xF0\x9F\x98\x96','U+1F616','CONFOUNDED FACE','sad','negative'],
                              [17,'\xF0\x9F\x98\x98','U+1F618','FACE THROWING A KISS','happy','positive'],
                              [18,'\xF0\x9F\x98\x9A','U+1F61A','KISSING FACE WITH CLOSED EYES','happy','positive'],
                              [19,'\\xF0\x9F\x98\x9C','U+1F61C','FACE WITH STUCK-OUT TONGUE AND WINKING EYE','happy','sarcastic'],
                              [20,'\xF0\x9F\x98\x9D','U+1F61D','FACE WITH STUCK-OUT TONGUE AND TIGHTLY-CLOSED EYES','happy','positive'],
                              [21,'\xF0\x9F\x98\x9E','U+1F61E','DISAPPOINTED FACE','sad','negative'],
                              [22,'\xF0\x9F\x98\xA0','U+1F620','ANGRY FACE','angry','negative'],
                              [23,'\xF0\x9F\x98\xA1','U+1F621','POUTING FACE','angry','negative'],
                              [24,'\xF0\x9F\x98\xA2','U+1F622','CRYING FACE','sad','negative'],
                              [25,'\xF0\x9F\x98\xA3','U+1F623','PERSEVERING FACE','sad','negative'],
                              [26,'\xF0\x9F\x98\xA4','U+1F624','FACE WITH LOOK OF TRIUMPH','angry','negative'],
                              [27,'\xF0\x9F\x98\xA5','U+1F625','DISAPPOINTED BUT RELIEVED FACE','sad','negative'],
                              [28,'\xF0\x9F\x98\xA8','U+1F628','FEARFUL FACE','sad','negative'],
                              [29,'\xF0\x9F\x98\xA9','U+1F629','WEARY FACE','sad','negative'],
                              [30,'\xF0\x9F\x98\xAA','U+1F62A','SLEEPY FACE','neutral','negative'],
                              [31,'\xF0\x9F\x98\xAB','U+1F62B','TIRED FACE','sad','negative'],
                              [32,'\xF0\x9F\x98\xAD','U+1F62D','LOUDLY CRYING FACE','sad','negative'],
                              [33,'\xF0\x9F\x98\xB0','U+1F630','FACE WITH OPEN MOUTH AND COLD SWEAT','sad','negative'],
                              [34,'\xF0\x9F\x98\xB1','U+1F631','FACE SCREAMING IN FEAR','sad','negative'],
                              [35,'\xF0\x9F\x98\xB2','U+1F632','ASTONISHED FACE','sad','negative'],
                              [36,'\xF0\x9F\x98\xB3','U+1F633','FLUSHED FACE','neutral','neutral'],
                              [37,'\xF0\x9F\x98\xB5','U+1F635','DIZZY FACE','sad','negative'],
                              [38,'\xF0\x9F\x98\xB7','U+1F637','FACE WITH MEDICAL MASK','sad','negative'],
                              [39,'\xF0\x9F\x98\xB8','U+1F638','GRINNING CAT FACE WITH SMILING EYES','happy','positive'],
                              [40,'\xF0\x9F\x98\xB9','U+1F639','CAT FACE WITH TEARS OF JOY','happy','positive'],
                              [41,'\xF0\x9F\x98\xBA','U+1F63A','SMILING CAT FACE WITH OPEN MOUTH','happy','positive'],
                              [42,'\xF0\x9F\x98\xBB','U+1F63B','SMILING CAT FACE WITH HEART-SHAPED EYES','happy','positive'],
                              [43,'\xF0\x9F\x98\xBC','U+1F63C','CAT FACE WITH WRY SMILE','angry','negative'],
                              [44,'\xF0\x9F\x98\xBD','U+1F63D','KISSING CAT FACE WITH CLOSED EYES','happy','positive'],
                              [45,'\xF0\x9F\x98\xBE','U+1F63E','POUTING CAT FACE','angry','negative'],
                              [46,'\xF0\x9F\x98\xBF','U+1F63F','CRYING CAT FACE','sad','negative'],
                              [47,'\xF0\x9F\x99\x80','U+1F640','WEARY CAT FACE','sad','negative'],
                              [48,'\xF0\x9F\x99\x85','U+1F645','FACE WITH NO GOOD GESTURE','sad','negative'],
                              [49,'\xF0\x9F\x99\x86','U+1F646','FACE WITH OK GESTURE','happy','positive'],
                              [50,'\xF0\x9F\x99\x87','U+1F647','PERSON BOWING DEEPLY','neutral','neutral'],         
                              [51,'\xF0\x9F\x99\x8B','U+1F64B','HAPPY PERSON RAISING ONE HAND','happy','positive'],
                              [52,'\xF0\x9F\x98\xB5','U+1F635','PERSON RAISING BOTH HANDS IN CELEBRATION','happy','positive'],
                              [53,'\xF0\x9F\x91\x86','U+1F446','WHITE UP POINTING BACKHAND INDEX','neutral','positive'],
                              [54,'\xF0\x9F\x91\x87','U+1F447','WHITE DOWN POINTING BACKHAND INDEX','neutral','positive'],
                              [55,'\xF0\x9F\x91\x88','U+1F448','WHITE LEFT POINTING BACKHAND INDEX','neutral','positive'],
                              [56,'\xF0\x9F\x91\x89','U+1F449','WHITE RIGHT POINTING BACKHAND INDEX','neutral','positive'],
                              [57,'\xF0\x9F\x91\x8A','U+1F44A','FISTED HAND SIGN','angry','negative'],
                              [58,'\xF0\x9F\x91\x8B','U+1F44B','WAVING HAND SIGN','sad','negative'],
                              [59,'\xF0\x9F\x91\x8C','U+1F44C','OK HAND SIGN','happy','positive'],
                              [60,'\xF0\x9F\x91\x8D','U+1F44D','THUMBS UP SIGN','happy','positive'],
                              [61,'\xF0\x9F\x91\x8E','U+1F44E','THUMBS DOWN SIGN','sad','negative'],
                              [62,'\xF0\x9F\x91\x8F','U+1F44F','CLAPPING HANDS SIGN','happy','positive'],
                              [63,'\xF0\x9F\x91\x90','U+1F450','OPEN HANDS SIGN','happy','positive'],
                              [64,'\xF0\x9F\x98\x80','U+1F600','GRINNING FACE','happy','positive'],
                              [65,'\xF0\x9F\x98\x87','U+1F607','SMILING FACE WITH HALO','happy','positive'],
                              [66,'\xF0\x9F\x98\x88','U+1F608','SMILING FACE WITH HORNS','happy','positive'],
                              [67,'\xF0\x9F\x98\x8E','U+1F60E','SMILING FACE WITH SUNGLASSES','happy','positive'],
                              [68,'\xF0\x9F\x98\x90','U+1F610','NEUTRAL FACE','neutral','neutral'],
                              [69,'\xF0\x9F\x98\x91','U+1F611','EXPRESSIONLESS FACE','neutral','negative'],
                              [70,'\xF0\x9F\x98\x95','U+1F615','CONFUSED FACE','sad','negative'],
                              [71,'\xF0\x9F\x98\x97','U+1F617','KISSING FACE','happy','positive'],
                              [72,'\xF0\x9F\x98\x99','U+1F619','KISSING FACE WITH SMILING EYES','happy','positive'],
                              [73,'\xF0\x9F\x98\x9B','U+1F61B','FACE WITH STUCK-OUT TONGUE','happy','sarcastic'],
                              [74,'\xF0\x9F\x98\x9F','U+1F61F','WORRIED FACE','sad','negative'],
                              [75,'\xF0\x9F\x98\xA6','U+1F626','FROWNING FACE WITH OPEN MOUTH','angry','negative'],
                              [76,'\xF0\x9F\x98\xA7','U+1F627','ANGUISHED FACE','sad','negative'],
                              [77,'\xF0\x9F\x98\xAC','U+1F62C','GRIMACING FACE','happy','positive'],
                              [78,'\xF0\x9F\x98\xAE','U+1F62E','FACE WITH OPEN MOUTH','neutral','positive'],
                              [79,'\xF0\x9F\x98\xAF','U+1F62F','HUSHED FACE','neutral','neutral'],
                              [80,'\xF0\x9F\x98\xB4','U+1F634','SLEEPING FACE','neutral','negative'],
                              [81,'\xF0\x9F\x98\xB6','U+1F636 ','FACE WITHOUT MOUTH','neutral','neutral']]
        
    
        s1=a
        #s2="\U0001"
        s2="\xF0\x9F"
        while (len(s1)>10 and s1.find(s2)>-1 ):
            s3= s1[s1.index(s2):]
            emocode.append(s3[:4])
            s1=s3[4:]
        #return Row(tweet=l[3], emojistatus=0, meaningtextemo=textemo, meaningtext=text, parttwo=textemoparttwo, emolist=emocode, abcd=a)
        statusemo=len(emocode)
        ctrp=0
        ctrn=0
        for pnval in emocode:
            if pnval=='positive':
                ctrp+=1
            elif pnval=='negative':
                ctrn+=1
        outset=list(set(emocode))
        if(statusemo==0):
            meanEmo.append("neutral")
            posorneg.append("neutral")
        else:
            for emoeach in outset:
                for uemo in range(len(emoticonsoveralllist)):
                    if emoeach==emoticonsoveralllist[uemo][1]:
                        meanEmo.append(emoticonsoveralllist[uemo][4])
                        posorneg.append(emoticonsoveralllist[uemo][5])
                        break
        return Row(tweet=str(text), tid=sid,status=0, emojistatus=len(outset), emoticonlist=emocode,emoticonMeaning=str(meanEmo), outputEmoSet=outset, outputposorneg=posorneg, totalpositive=ctrp, totalnegative=ctrn )
    else:
        return Row(tweet="not there",tid=0,status=0, emojistatus=0, emoticonlist=[],emoticonMeaning=[], outputEmoSet=[], outputposorneg=["null"],totalpositive=0, totalnegative=0)

In [4]:
#slang dict creation
f = open('slang.txt', 'r')
slang_dict = {}
for line in f:
    line = line.strip()
    parts = [p.strip() for p in line.split("=")]
    slang_dict[parts[0].lower()] = (parts[1].lower())
f.close()

In [5]:
positive = list()
f = open('positive.txt', 'r')
for line in f:
    line = line.strip()
    positive.append(line)
f.close()

In [6]:
negative = list()
f = open('negative.txt', 'r')
for line in f:
    line = line.strip()
    negative.append(line)
f.close()

In [7]:
sys.path.append('C:\Python27\Lib\site-packages')
import enchant
import editdistance 
d = enchant.Dict("en_US")
sqlContext = SQLContext(sc)
import re ,string
def preprocess(x):
    text=str(x[9])
    sid=x[6]
    emocount=x[0]
    emoticon_posneg=x[4]
    t_neg=x[7]
    t_pos=x[8]
    #convert to lower case
    text=text.lower()
    
    #2)standardising slang words
    
    l1 = text.split(" ")
    if '' in l1:
        l1.remove('')
    for i in range(len(l1)):
        if l1[i] in slang_dict:
             l1[i] = slang_dict[l1[i]]
    text=' '.join(l1)
     
    #covert @user to at_user
    text= re.sub('@[^\s]+','atuser',text)
    
    text=text.replace('sarcas', '')
      
    #covert #data to data
    #text = re.sub(r'#([^\s]+)', r'\1', text)
    #Remove additional white spaces
    text = re.sub('[\s]+', ' ', text)
    
    text  = text.replace('\\', '')
    #replace url by 'url'
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','url',text)
    
    #--feature vector--#
    #1)removing punctuation marks
    text =  text.translate(string.maketrans("",""), string.punctuation)
    #3) removing repeated chars
    pattern = re.compile(r"(.)\1{1,}", re.DOTALL)
    text = pattern.sub(r"\1\1", text)
    
    #3) removing words containing no
    text=re.sub(r'\w*\d\w*', '', text).strip()
    
    return Row(tweet=str(text),tid=sid, status=0, emoticon_postivenegative=emoticon_posneg,emo_count=emocount, total_neg=t_neg,total_pos=t_pos)

In [8]:
rdd1 = sc.textFile('combined_new.txt')
rdd1.cache()
rdd1.collect()

[u'{"created_at":"Tue Jan 24 15:01:38 +0000 2017","id":823908626423169025,"id_str":"823908626423169025","text":"Even my task doesn\'t fail successfully, its like- \\"I... I mean... Do you wanna... Aah... Where\'s bathroom ma\'am\\"\\ud83d\\ude02\\u2026 https:\\/\\/t.co\\/FF5PWCqcRH","display_text_range":[0,140],"source":"\\u003ca href=\\"http:\\/\\/www.twitter.com\\" rel=\\"nofollow\\"\\u003eTwitter for Windows\\u003c\\/a\\u003e","truncated":true,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":733404929277067264,"id_str":"733404929277067264","name":"The Memeist","screen_name":"TheMemeist","location":"Jaipur Rajasthan India ","url":null,"description":"Taking Interesting memes from around the world, The Memeist is here with his bag of memes. #Comedy #memes .\\rFollow us on: Fb @TheMemeist \\/ Insta @ImTheMemeist","protected":false,"verified":false,"followers_count":10,"frie

In [9]:
final = rdd1.map(fil)
#print final.count()
final1= final.filter(lambda x: x!=None)
#print final1.count()
final1.collect()

[Row(emojistatus=0, emoticonMeaning="['neutral']", emoticonlist=[], outputEmoSet=[], outputposorneg=['neutral'], status=0, tid=823908626423169025L, totalnegative=0, totalpositive=0, tweet='"Even my task doesn\'t fail successfully'),
 Row(emojistatus=0, emoticonMeaning=[], emoticonlist=[], outputEmoSet=[], outputposorneg=['null'], status=0, tid=0, totalnegative=0, totalpositive=0, tweet='not there'),
 Row(emojistatus=0, emoticonMeaning="['neutral']", emoticonlist=[], outputEmoSet=[], outputposorneg=['neutral'], status=0, tid=823911134268772352L, totalnegative=0, totalpositive=0, tweet='"\\"I like money and it\'s green.\\" #sarcasm https:\\/\\/t.co\\/Sat3CyEvLG"'),
 Row(emojistatus=0, emoticonMeaning=[], emoticonlist=[], outputEmoSet=[], outputposorneg=['null'], status=0, tid=0, totalnegative=0, totalpositive=0, tweet='not there'),
 Row(emojistatus=0, emoticonMeaning="['neutral']", emoticonlist=[], outputEmoSet=[], outputposorneg=['neutral'], status=0, tid=823916522234150912L, totalnegat

In [10]:
preprocessed_rdd = final1.map(preprocess)
preprocessed_rdd.count()
preprocessed_rdd.collect()

[Row(emo_count=0, emoticon_postivenegative=['neutral'], status=0, tid=823908626423169025L, total_neg=0, total_pos=0, tweet='even my task doesnt fail successfully'),
 Row(emo_count=0, emoticon_postivenegative=['null'], status=0, tid=0, total_neg=0, total_pos=0, tweet='not there'),
 Row(emo_count=0, emoticon_postivenegative=['neutral'], status=0, tid=823911134268772352L, total_neg=0, total_pos=0, tweet='i like money and its green m url'),
 Row(emo_count=0, emoticon_postivenegative=['null'], status=0, tid=0, total_neg=0, total_pos=0, tweet='not there'),
 Row(emo_count=0, emoticon_postivenegative=['neutral'], status=0, tid=823916522234150912L, total_neg=0, total_pos=0, tweet='lol aww'),
 Row(emo_count=0, emoticon_postivenegative=['null'], status=0, tid=0, total_neg=0, total_pos=0, tweet='not there'),
 Row(emo_count=0, emoticon_postivenegative=['neutral'], status=0, tid=823916631579574272L, total_neg=0, total_pos=0, tweet='rt atuser lol aww'),
 Row(emo_count=0, emoticon_postivenegative=['nu

In [11]:
## remove the undesired tweets
def rem_null(x):
    #tweet = x[0]
    #if x[4] != "not there" and len(tweet.split(' '))>3:
    if x[6] != "not there":
        return x

In [12]:
preprocessed_rdd = preprocessed_rdd.filter(rem_null)
preprocessed_rdd.count()
#preprocessed_rdd.collect()

11922

In [13]:
def rem_noemo(x):
    if x[0]>=1:
        return x
    

In [14]:
preprocessed_rdd=preprocessed_rdd.filter(rem_noemo)
preprocessed_rdd.count()

894

In [15]:
def check_word(tweet,word):
    t_list = tweet.split(' ')
    w_list = word.split(' ')
    l = len(w_list)
    for i in range(0,len(t_list)-l+1):
        if t_list[i:i+l] == w_list:
            return 1
            break
        else:
            pass
    return 0
            
    

In [16]:
###positive sarcastic

In [17]:
def filter_pos_sar(x):
    tweet = x[6]
    stat = x[2]
    sid = x[3]
    
    for term in positive:
        if(check_word(tweet,term)==1):
            stat = 1
            break
    return(Row(status=stat,tid=sid,tweet=tweet))

In [18]:
pos_sarcastic = preprocessed_rdd.map(filter_pos_sar)
pos_sarcastic.count()

894

In [19]:
##negative sarcastic

In [20]:
def filter_neg_sar(x):
    tweet = x[6]
    stat = x[2]
    sid = x[3]
    for term in negative:
        if(check_word(tweet,term)==1):
            stat = 1
            break
    return(Row(status=stat,tid=sid,tweet=tweet))
    

In [21]:
neg_sarcastic = preprocessed_rdd.map(filter_pos_sar)
neg_sarcastic.count()

894

In [22]:
##unordered positive neg

In [23]:
def filter_unordered_sar(x):
    tweet = x[6]
    stat = x[2]
    sid = x[3]

    for term in positive:
        if(check_word(tweet,term)==1):
            for term in negative:
                if(check_word(tweet,term)==1):
                    stat=1
                    break
    return(Row(status=stat,tid=sid,tweet=tweet))
            

In [24]:
unordered_sarcastic = preprocessed_rdd.filter(filter_unordered_sar)
unordered_sarcastic.count()

894

In [25]:
##only positive

In [26]:
def only_pos(x):
    tweet = x[6]
    stat = x[2]
    sid = x[3]
    emostat=x[1]

    for term in positive:
        if(check_word(tweet,term)==1):
            stat=1
            for term in negative:
                if(check_word(tweet,term)==1):
                    stat=0
                    break                
    return(Row(tweet_status=stat,tid=sid,tweet=tweet, emostatus=emostat))
            

In [27]:
only_pos_tweets = preprocessed_rdd.map(only_pos)
only_pos_tweets.count()
#only_pos_tweets.collect()

894

In [28]:
def filter_only_pos(x):
    if x[3]==1:
        return x

In [29]:
only_pos_tweets = only_pos_tweets.filter(filter_only_pos)
only_pos_tweets.count()


518

In [30]:
##only negative

In [31]:
def only_neg(x):
    tweet = x[6]
    stat = x[2]
    sid = x[3]
    emostat=x[1]

    for term in negative:
        if(check_word(tweet,term)==1):
            stat=1
            for term in positive:
                if(check_word(tweet,term)==1):
                    stat=0
                    break                
    return(Row(neg_tweet_status=stat,tid=sid,tweet=tweet,emostatus=emostat))
            

In [32]:
only_neg_tweets = preprocessed_rdd.map(only_neg)
only_neg_tweets.count()

894

In [33]:
def filter_only_neg(x):
    if x[3]==1:
        return x

In [34]:
only_neg_tweets = only_neg_tweets.filter(filter_only_neg)
only_neg_tweets.count()

0

In [35]:
##sarcasm_detection

In [36]:
def sarcasm_detection(x):
    tweet = x[6]
    stat = x[2]
    sid = x[3]
    emostat=x[1]
    setemostat=[]
    t_pos_emo=x[5]
    t_neg_emo=x[4]
    sarc=''

    for term in positive:
        if(check_word(tweet,term)==1):
            stat=1
            for term in negative:
                if(check_word(tweet,term)==1):
                    stat=0
                    break 
    for term in negative:
        if(check_word(tweet,term)==1):
            stat=2
            for term in positive:
                if(check_word(tweet,term)==1):
                    stat=0
                    break 
    setemostat=list(set(emostat))
    ctrp=0
    ctrn=0
    for pnval in emostat:
        if pnval=='positive':
            ctrp+=1
        elif pnval=='negative':
            ctrn+=1
            

        
    if len(setemostat)==1:
        emo=setemostat[0]
        if(emo=='positive' and stat==2):
            sarc='sarcastic'
        elif(emo=='negative' and stat==1):
            sarc='sarcastic'
        elif(emo=='sarcastic'):
            sarc='sarcastic'
        else:
            sarc='nonsarcastic'
    else:
        sarc='nonsarcastic'
    #if 'sarcas' in tweet:
    #    sarc='sarcastic'
    #   s_in_tweet=1
    #else:
    s_in_tweet=0
        
    #return(Row(tweetstatus=stat,tid=sid,tweet=tweet, emostatus=setemostat, sarcastic_status=sarc, total_pos_emo=ctrp, total_neg_emo=ctrn, sarcastic_in_tweet=s_in_tweet))
    return(Row(tweetstatus=stat,tid=sid,tweet=tweet, emostatus=setemostat, sarcastic_status=sarc, total_pos_emo=ctrp, total_neg_emo=ctrn, s=s_in_tweet))        

In [37]:
sarcastic_tweets_data = preprocessed_rdd.map(sarcasm_detection)
sarcastic_tweets_data.count()
sarcastic_tweets_data.collect()

[Row(emostatus=[], s=0, sarcastic_status='nonsarcastic', tid=823983257624711171L, total_neg_emo=0, total_pos_emo=0, tweet='like hes realnlive like you know better  biblical  url', tweetstatus=1),
 Row(emostatus=['negative'], s=0, sarcastic_status='sarcastic', tid=824070607084068869L, total_neg_emo=1, total_pos_emo=0, tweet='a shorty for pleky  god', tweetstatus=1),
 Row(emostatus=['positive'], s=0, sarcastic_status='nonsarcastic', tid=824669978574123008L, total_neg_emo=0, total_pos_emo=1, tweet='atuser hait means laughing  surely might get a discount to join premium acc tic', tweetstatus=1),
 Row(emostatus=['positive'], s=0, sarcastic_status='sarcastic', tid=825259403154595840L, total_neg_emo=0, total_pos_emo=1, tweet='for every mishaps blame  religion m', tweetstatus=2),
 Row(emostatus=['positive'], s=0, sarcastic_status='nonsarcastic', tid=825263994298630145L, total_neg_emo=0, total_pos_emo=1, tweet='friend asking what shall i do with my american passport me burn it m', tweetstatus=0

In [38]:
def filter_sar(x):
    if x!=None:
        if x[2]=='sarcastic':
            return x

In [39]:
sarcastic_tweets = sarcastic_tweets_data.filter(filter_sar)
sarcastic_tweets.count()

123

In [40]:
sarcastic_tweets.collect()

[Row(emostatus=['negative'], s=0, sarcastic_status='sarcastic', tid=824070607084068869L, total_neg_emo=1, total_pos_emo=0, tweet='a shorty for pleky  god', tweetstatus=1),
 Row(emostatus=['positive'], s=0, sarcastic_status='sarcastic', tid=825259403154595840L, total_neg_emo=0, total_pos_emo=1, tweet='for every mishaps blame  religion m', tweetstatus=2),
 Row(emostatus=['sarcastic'], s=0, sarcastic_status='sarcastic', tid=825274385447661568L, total_neg_emo=0, total_pos_emo=0, tweet='where losers are fighting to decide who is the biggest loser and that loser will be the winner of  m justsaying', tweetstatus=1),
 Row(emostatus=['sarcastic'], s=0, sarcastic_status='sarcastic', tid=825287691952590849L, total_neg_emo=0, total_pos_emo=0, tweet='atuser what he was going to say before cucked out  m url', tweetstatus=1),
 Row(emostatus=['sarcastic'], s=0, sarcastic_status='sarcastic', tid=825329125271470080L, total_neg_emo=0, total_pos_emo=0, tweet='hey my lover  find me by name  url url', tweet

In [41]:
def filter_non_sar(x):
    if x!=None:
        if x[2]=='nonsarcastic':
            return x

In [42]:
non_sarcastic_tweets = sarcastic_tweets_data.filter(filter_non_sar)
non_sarcastic_tweets.count()

771

In [43]:
non_sarcastic_tweets.collect()

[Row(emostatus=[], s=0, sarcastic_status='nonsarcastic', tid=823983257624711171L, total_neg_emo=0, total_pos_emo=0, tweet='like hes realnlive like you know better  biblical  url', tweetstatus=1),
 Row(emostatus=['positive'], s=0, sarcastic_status='nonsarcastic', tid=824669978574123008L, total_neg_emo=0, total_pos_emo=1, tweet='atuser hait means laughing  surely might get a discount to join premium acc tic', tweetstatus=1),
 Row(emostatus=['positive'], s=0, sarcastic_status='nonsarcastic', tid=825263994298630145L, total_neg_emo=0, total_pos_emo=1, tweet='friend asking what shall i do with my american passport me burn it m', tweetstatus=0),
 Row(emostatus=['positive'], s=0, sarcastic_status='nonsarcastic', tid=825289973729488898L, total_neg_emo=0, total_pos_emo=1, tweet='atuser nnso true but at least i was able to get out early and enjoy the fantastic weather  m crapbritishweather', tweetstatus=0),
 Row(emostatus=[], s=0, sarcastic_status='nonsarcastic', tid=825296562096111616L, total_ne

In [44]:
def filter_multi_emostatus(x):
    if x!=None:
        if len(x[0])>1 and x[1]==0:
            return x

In [45]:
multiemo_tweets = sarcastic_tweets_data.filter(filter_multi_emostatus)
multiemo_tweets.count()

37

In [46]:
multiemo_tweets.collect()

[Row(emostatus=['positive', 'negative'], s=0, sarcastic_status='nonsarcastic', tid=825347498797629440L, total_neg_emo=1, total_pos_emo=1, tweet='lol funny comedy m rt weneedfeminlsm i love zendaya  url', tweetstatus=0),
 Row(emostatus=['positive', 'negative'], s=0, sarcastic_status='nonsarcastic', tid=825415536376942592L, total_neg_emo=1, total_pos_emo=1, tweet='amazing christian values  m url', tweetstatus=0),
 Row(emostatus=['positive', 'negative'], s=0, sarcastic_status='nonsarcastic', tid=825616546047201280L, total_neg_emo=1, total_pos_emo=1, tweet='so i just showed my roti flipping skills to my  now my family wants me to get a degree in hotel', tweetstatus=2),
 Row(emostatus=['positive', 'negative'], s=0, sarcastic_status='nonsarcastic', tid=825705275873062915L, total_neg_emo=1, total_pos_emo=1, tweet='sometimes sorry means nothing  pleaseread m url', tweetstatus=0),
 Row(emostatus=['positive', 'negative'], s=0, sarcastic_status='nonsarcastic', tid=826104502252138496L, total_neg_e

In [47]:
def assign_to_multi_emostatus(x):
    e_stat=x[0]
    t_stat=x[7]
    s_stat=x[2]
    e_pos=x[5]
    e_neg=x[4]
    sid=x[3]
    tweet=x[6]
    s_in_tweet=x[1]
    
    emo_stat=[]
    if 'sarcastic' in e_stat:
        s_stat='sarcastic'
        emo_stat.append('sarcastic')
    else:
        if(e_pos>e_neg):
            emo_stat.append('positive')
        elif(e_pos<e_neg):
            emo_stat.append('negative')
        else:
            emo_stat=e_stat
    
    if len(emo_stat)==1:
        emo=emo_stat[0]
        if(emo=='positive' and t_stat==2):
            sarc='sarcastic'
        elif(emo=='negative' and t_stat==1):
            sarc='sarcastic'
        elif(emo=='sarcastic'):
            sarc='sarcastic'
        else:
            sarc='nonsarcastic'
    else:
        sarc='nonsarcastic'
        
    return(Row(tweetstatus=t_stat,tid=sid,tweet=tweet, emostatus=emo_stat, sarcastic_status=sarc, total_pos_emo=e_pos, total_neg_emo=e_neg, s=s_in_tweet))
        
    
    
    
    

In [48]:
multiemo_tweets_data = multiemo_tweets.map(assign_to_multi_emostatus)
multiemo_tweets_data.count()
multiemo_tweets_data.collect()

[Row(emostatus=['positive', 'negative'], s=0, sarcastic_status='nonsarcastic', tid=825347498797629440L, total_neg_emo=1, total_pos_emo=1, tweet='lol funny comedy m rt weneedfeminlsm i love zendaya  url', tweetstatus=0),
 Row(emostatus=['positive', 'negative'], s=0, sarcastic_status='nonsarcastic', tid=825415536376942592L, total_neg_emo=1, total_pos_emo=1, tweet='amazing christian values  m url', tweetstatus=0),
 Row(emostatus=['positive', 'negative'], s=0, sarcastic_status='nonsarcastic', tid=825616546047201280L, total_neg_emo=1, total_pos_emo=1, tweet='so i just showed my roti flipping skills to my  now my family wants me to get a degree in hotel', tweetstatus=2),
 Row(emostatus=['positive', 'negative'], s=0, sarcastic_status='nonsarcastic', tid=825705275873062915L, total_neg_emo=1, total_pos_emo=1, tweet='sometimes sorry means nothing  pleaseread m url', tweetstatus=0),
 Row(emostatus=['negative'], s=0, sarcastic_status='sarcastic', tid=826104502252138496L, total_neg_emo=2, total_pos

In [49]:
multiemo_sarcastic_tweets = multiemo_tweets_data.filter(filter_sar)
multiemo_sarcastic_tweets.count()

10

In [50]:
multiemo_sarcastic_tweets.collect()

[Row(emostatus=['negative'], s=0, sarcastic_status='sarcastic', tid=826104502252138496L, total_neg_emo=2, total_pos_emo=1, tweet='the day you turned into something you said youd never be tsk  sayitwhenyoumeanit', tweetstatus=1),
 Row(emostatus=['sarcastic'], s=0, sarcastic_status='sarcastic', tid=830106894001922049L, total_neg_emo=0, total_pos_emo=1, tweet='atuser point for you i had to think about the comments in your picture books lots of books  m', tweetstatus=0),
 Row(emostatus=['negative'], s=0, sarcastic_status='sarcastic', tid=830779974516891649L, total_neg_emo=1, total_pos_emo=0, tweet='rt atuser looking forward to cleaning tack tomorrow  m atuser', tweetstatus=1),
 Row(emostatus=['negative'], s=0, sarcastic_status='sarcastic', tid=830793840567029760L, total_neg_emo=3, total_pos_emo=1, tweet='heynheynobviouslynyou dont knowncpc jesus orngopjesus  url', tweetstatus=1),
 Row(emostatus=['sarcastic'], s=0, sarcastic_status='sarcastic', tid=838173931970383872L, total_neg_emo=1, tota

In [51]:
multiemo_non_sarcastic_tweets = multiemo_tweets_data.filter(filter_non_sar)
multiemo_non_sarcastic_tweets.count()

27

In [52]:
multiemo_non_sarcastic_tweets.collect()

[Row(emostatus=['positive', 'negative'], s=0, sarcastic_status='nonsarcastic', tid=825347498797629440L, total_neg_emo=1, total_pos_emo=1, tweet='lol funny comedy m rt weneedfeminlsm i love zendaya  url', tweetstatus=0),
 Row(emostatus=['positive', 'negative'], s=0, sarcastic_status='nonsarcastic', tid=825415536376942592L, total_neg_emo=1, total_pos_emo=1, tweet='amazing christian values  m url', tweetstatus=0),
 Row(emostatus=['positive', 'negative'], s=0, sarcastic_status='nonsarcastic', tid=825616546047201280L, total_neg_emo=1, total_pos_emo=1, tweet='so i just showed my roti flipping skills to my  now my family wants me to get a degree in hotel', tweetstatus=2),
 Row(emostatus=['positive', 'negative'], s=0, sarcastic_status='nonsarcastic', tid=825705275873062915L, total_neg_emo=1, total_pos_emo=1, tweet='sometimes sorry means nothing  pleaseread m url', tweetstatus=0),
 Row(emostatus=['positive', 'negative'], s=0, sarcastic_status='nonsarcastic', tid=826783003355521024L, total_neg_e